In [1]:

import numpy as np
import pandas as pd
from scipy.stats import rankdata

matrix = np.array([
    [0.134, 0.140, 0.160, 0.140],
    [0.199, 0.105, 0.304, 0.104],
    [0.156, 0.093, 0.061, 0.084],
    [0.021, 0.128, 0.050, 0.128],
    [0.103, 0.185, 0.041, 0.206],
    [0.178, 0.152, 0.157, 0.116],
    [0.021, 0.128, 0.069, 0.091],
    [0.188, 0.070, 0.160, 0.131]
])

sites = ['Site1', 'Site2', 'Site3', 'Site4', 'Site5', 'Site6', 'Site7', 'Site8']
criteria = ['C1', 'C2', 'C3', 'C4']
criteria_types = ['max', 'max', 'max', 'max']
weights = np.array([0.15288888, 0.30888675, 0.45773133, 0.080493041])

# NORMALIZATION FUNCTION (min-max normalization)
def normalize_matrix(matrix, criteria_types):
    norm_matrix = np.zeros_like(matrix, dtype=float)
    for j in range(matrix.shape[1]):
        col = matrix[:, j]
        if criteria_types[j] == 'max':
            norm_matrix[:, j] = (col - np.min(col)) / (np.max(col) - np.min(col))
        else:  # 'min'
            norm_matrix[:, j] = (np.max(col) - col) / (np.max(col) - np.min(col))
    return norm_matrix

# CODAS METHOD
def codas(matrix, weights, criteria_types, sites, criteria, tau=0.02):
    norm_matrix = normalize_matrix(matrix, criteria_types)
    weighted_matrix = norm_matrix * weights

    # Negative-ideal solution (worst for each criterion)
    neg_ideal = np.min(weighted_matrix, axis=0)

    # Euclidean and Taxicab distances from negative-ideal
    euclidean = np.sqrt(((weighted_matrix - neg_ideal) ** 2).sum(axis=1))
    taxicab = np.abs(weighted_matrix - neg_ideal).sum(axis=1)

    # Assessment score
    # If difference in Euclidean distance between two alternatives < tau, use Taxicab distance as tie-breaker
    scores = np.zeros(len(sites))
    for i in range(len(sites)):
        count = 0
        for j in range(len(sites)):
            if i == j:
                continue
            diff = euclidean[i] - euclidean[j]
            if diff > tau:
                count += 1
            elif abs(diff) <= tau:
                if taxicab[i] > taxicab[j]:
                    count += 1
        scores[i] = count

    # Alternatively, for ranking, you can use the Euclidean distance directly (higher is better)
    df = pd.DataFrame({
        'Euclidean': euclidean,
        'Taxicab': taxicab,
        'CODAS Score': euclidean,  # For simple ranking, use Euclidean distance
        'Rank': rankdata(-euclidean, method='min')
    }, index=sites).sort_values('Rank')

    return df

result_df = codas(matrix, weights, criteria_types, sites, criteria)
print(result_df)


       Euclidean   Taxicab  CODAS Score  Rank
Site2   0.491838  0.717825     0.491838     1
Site6   0.328481  0.578103     0.328481     2
Site5   0.326880  0.459812     0.326880     3
Site1   0.298381  0.529135     0.298381     4
Site8   0.253834  0.381561     0.253834     5
Site7   0.163296  0.209137     0.163296     6
Site4   0.159240  0.200480     0.159240     7
Site3   0.135918  0.212541     0.135918     8
